# Firework Algorithm for Optimization

- Autores originais: [*Ying Tan*](https://scholar.google.com/citations?user=PjNxSPsAAAAJ&hl=en) e [*Yuanchun Zhu*](https://ieeexplore.ieee.org/author/38008573100)
- Ano de Publicação: 2010
- Editora: [Springer](https://link.springer.com/chapter/10.1007/978-3-642-13495-1_44)
- Conferência: [Advances in Swarm Intelligence](https://link.springer.com/book/10.1007/978-3-642-13495-1): First International Conference, ICSI 2010, Beijing, China, June 12-15, 2010, Proceedings, Part I 1

## Descrição:
FA é um algoritmo de inteligência de enxames inspirado na explosão de fogos de artifício. Esse algoritmo é utilizado para a otimização global de funções complexas. São fornecidos dois processos de *“explosão”* (busca) e os mecanismos para manter a diversidade das *“faíscas”*.

In [2]:
import numpy as np
import pandas as pd
import random

In [3]:
def objective_function(x):
    result = x**2
    return result

In [4]:
value = random.gauss(1,1)
value

-0.5131434364071157

In [5]:
value = random.uniform(-1,1)
value

0.38152161999371925

### Inicializando posições dos Fogos de Artifício:

In [6]:
def initialize_firework_locations(n, xmin, xmax):
    fireworks = xmin + (xmax - xmin) * np.random.rand(n)
    return fireworks

In [7]:
fireworks = initialize_firework_locations(5, -5.12, 5.12)
fireworks

array([ 4.62007573,  1.9749855 ,  2.94385069,  1.80464276, -4.35520061])

### Calculando o número de faíscas:

In [8]:
def sparks_number(fx, m, ymax, eps=.00001):
    sum_difference_of_worst_fit_and_fx = np.sum(ymax-fx) + eps
    s = m* ymax - fx + eps/sum_difference_of_worst_fit_and_fx
    s = firework_boundaries(0.1, 0.8, m, s)
    return s

In [9]:
def firework_boundaries(a, b, m, s):
    term1 = round(a*m) * (s < (a*m))
    term2 = round(b*m) * (s> b*m)
    term3 = np.round(s) * (~(s < (a*m)) & ~((s > b *m) & (a<b<1)))
    print(term1, term2, term3)
    s_new = term1 + term2 + term3
    return s_new

In [10]:
x = np.array([45.2,5.2,12.5])
fx = objective_function(x)
m = 10
ymax = x.max()
sparks_number(fx, m, ymax)

[1 0 0] [0 8 8] [-0.  0.  0.]


array([1., 8., 8.])

## Obtendo localização das Faíscas:

### Estratégia de Explosão:

In [11]:
def explosion_amplitude(A_hat, fx, ymin, eps=.00001):
    sum_difference_of_fx_and_best_fit = np.sum(fx - ymin) + eps
    A = A_hat * (fx - ymin + eps/ sum_difference_of_fx_and_best_fit)
    return A

In [12]:
d = 6
round(6 * random.uniform(0,1))

3

In [13]:
A_hat = 10
x = np.array([1.3, 5.2, 2.8])
fx = objective_function(x)
ymin = x.min()

explosion_amplitude(A_hat, fx, ymin)

array([  3.90000306, 257.40000306,  65.40000306])

In [14]:
def obtain_location_of_spark(d, amp, xmin, xmax):
    #selecting z dimensions to be affected
    xj = np.zeros(d)
    z = round(d * random.uniform(0,1))
    print("number of dimensions to be affected", z)
    random_dimensions = np.random.choice(d, z, replace=False)
    print("random dimensions selected", random_dimensions)
    h = amp * random.uniform(-1 ,1)
    print("displacement: ",h)
    for i in random_dimensions:
        xj[i] = 1
    
    random_affected_dimensions = xj * h
    #mapping affected dimensions to potential space
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

In [15]:
def map_to_potential_space(xmin, xmax, affected):
    print("xmax - xmin: ", xmax - xmin)
    inbound = xmin + abs(affected)%(xmax - xmin)
    print("inbound: ", inbound)
    print(inbound)
    term1 = (affected > xmax) * inbound
    print("values that are over the max bound",term1)
    term2 =  (affected < xmin) * inbound
    print("values that are over the min bound",term2)
    fixed_bounds = term1 + term2
    mapping = (fixed_bounds == 0) * affected + fixed_bounds 
    print("mapping: ",  mapping)
    return mapping

In [16]:
xmin = np.array([-3.12, -4.5, -6.6])
xmax = np.array([3.12, 4.5, 6.6])
d = 3
amp = 5
obtain_location_of_spark(d, amp, xmin, xmax)

number of dimensions to be affected 2
random dimensions selected [1 2]
displacement:  0.13903611671031046
xmax - xmin:  [ 6.24  9.   13.2 ]
inbound:  [-3.12       -4.36096388 -6.46096388]
[-3.12       -4.36096388 -6.46096388]
values that are over the max bound [-0. -0. -0.]
values that are over the min bound [-0. -0. -0.]
mapping:  [0.         0.13903612 0.13903612]


array([0.        , 0.13903612, 0.13903612])

### Estratégia de Faíscas Gaussianas:

In [ ]:
def gaussian_spark_location(xmin, xmax, d):
    return

In [17]:
np.random.choice(10, 10, replace=False)

array([3, 8, 1, 4, 7, 2, 6, 0, 9, 5])